In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

class_labels = {
    0: "LLWNGPMAV",
    1: "RPRGEVRFL",
    2: "ATDALMTGY",
    3: "HSKKKCDEL",
    4: "KAFSPEVIPMF",
    5: "KRWIILGLNK",
    6: "KRWIIMGLNK",
    7: "TPQDLNTML",
    8: "EIYKRWII",
    9: "ISPRTL-W",
    10: "FLKEKGGL",
    11: "HPKVSSEVHI",
    12: "IIKDYGKQM",
    13: "LPPIVAKEI",
    14: "RFPLTFGWCF",
    15: "RYPLTFGWCF",
    16: "TPGPGVRYPL",
    17: "TQGYFPDWQNY",
    18: "FPRPWLHGL",
    19: "RYPLTFGW",
    20: "ELRRKMMYM",
    21: "QIKVRVDMV",
    22: "QIKVRVKMV",
    23: "VLEETSVML",
    24: "FPTKDVAL",
    25: "NLVPMVATV",
    26: "RPHERNGFTVL",
    27: "TPRVTGGGAM",
    28: "VTEHDTLLY",
    29: "YLAMPFATPMEAELARRSLA",
    30: "GLCTLVAML",
    31: "YVLDHLIVV",
    32: "EPLPQGQLTAY",
    33: "RAKFKQLL",
    34: "HPVGEADYFEY",
    35: "FLRGRAYGL",
    36: "AVFDRKSDAK",
    37: "IVTDFSVIK",
    38: "NFIRMVISNPAAT",
    39: "KRGIVEQSSTSISSL",
    40: "ENPVVHFFKNIVTPR",
    41: "GILGFVFTL",
    42: "PQPELPYPQPE",
    43: "FWIDLFETIG",
    44: "other"
  }

tokenizer = AutoTokenizer.from_pretrained("wukevin/tcr-bert")
model = AutoModelForSequenceClassification.from_pretrained("wukevin/tcr-bert")

input_text = "I like you. I love you".upper()
inputs = tokenizer(input_text, return_tensors="pt")

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    
logits = outputs.logits
probabilities = torch.softmax(logits, dim=1)


result = list(zip(list(class_labels.values()), probabilities.tolist()[0]))
result.sort(key = lambda x: -x[1])
for word, probability in result:
    word = '\"' + word + '\"'
    print(f"P(Y = {word:25} | x = x) = {probability:.5f}")
    # print(f"Class: {word:30}| Probability: {probability:.4f}")

In [ ]:
model

In [21]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

class_labels = {0: "R", 1: "H", 2: "K", 3: "D", 4: "E", 5: "S", 6: "T", 7: "N", 8: "Q", 9: "C", 10: "U", 11: "G", 12: "P", 13: "A", 14: "V", 15: "I", 16: "L", 17: "M", 18: "F", 19: "Y", 20: "W", 21: "$", 22: ".", 23: "?", 24: "|", 25: "*"}

tokenizer = AutoTokenizer.from_pretrained("./model/mlm-only/tokenizer")
model = AutoModelForMaskedLM.from_pretrained("./model/mlm-only/model").bert
text = ["CATVSFTGANSKLTF", "CADNDYKLSF"]

input_text = [" ".join(list("." + i[1::])) for i in text]
inputs = tokenizer(input_text, return_tensors="pt", padding = True)

# Predict
with torch.no_grad():
    outputs = model(**inputs)
    
embeddings = outputs.last_hidden_state[:,-1,:]

In [42]:
import pandas as pd

pd.DataFrame(embeddings.tolist()).to_parquet("file.parquet")

In [44]:
pd.read_parquet("file.parquet").to_csv("file.csv")

In [ ]:
predictions = outputs.logits
mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
mask_token_logits = predictions[0, mask_token_index, :]

# Apply softmax to convert logits to probabilities
token_probabilities = torch.softmax(mask_token_logits, dim=1)

# Get the top 5 predicted tokens and their probabilities
top_5_tokens = torch.topk(token_probabilities, 5, dim=1)
top_5_token_ids = top_5_tokens.indices[0].tolist()
top_5_probabilities = top_5_tokens.values[0].tolist()

# Convert predicted token IDs to words and print them with probabilities
for token_id, probability in zip(top_5_token_ids, top_5_probabilities):
    word = tokenizer.decode([token_id])
    print(f"Predicted word: {word}, Probability: {probability:.4f}")

In [53]:
import os
from pathlib import Path
os.path.exists(Path("C:\\Users\\rcwyuen\OneDrive - University College London\\Academic\\Year 4 2023-2024\\Y4 COMP0138 Dissertation\\Code\\TCR-Cancer-Prediction\\data\\embeddings\\lung_cancer\\f.csv"))

False

In [63]:
p = Path("C:\\Users\\rcwyuen\OneDrive - University College London\\Academic\\Year 4 2023-2024\\Y4 COMP0138 Dissertation\\Code\\TCR-Cancer-Prediction\\data\\embeddings\\lung_cancer.csv")
p.parent.name

'embeddings'